In [ ]:
#################################
# Project 1: Face Mask Detector
# 
#         Group 2
# 
#       Mansur Rahman
#   Mikael Falk Lundgren
#       Yann Reibel
#################################

In [ ]:
# This sets up, creates and trains model 1 
import os
import torch, torchvision
from torch import nn
from pathlib import Path
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm
from typing import List, Tuple, Dict

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup path to data folder
data_path = Path("./data/")
image_path = data_path / "Face_Mask_Dataset"
# Setup train, testing and validation paths
train_dir = image_path / "Train"
test_dir = image_path / "Test"
val_dir = image_path / "Validation"

def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Traverses and prints directory details
walk_through_dir(image_path)

# Transform (with 1 data aug to improve performance) to tensor
data_transform = transforms.Compose([transforms.Resize(80), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])

# Create pytorch datasets for train, test and validation
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

# Checking the data and a sample
print(f"Train data:\n{train_data}\nTest data:\n{test_data}")
img_1, label_1 = train_data[0][0], train_data[0][1]
print(f"Image tensor:\n{img_1}")
print(f"Image shape: {img_1.shape}")
print(f"Image datatype: {img_1.dtype}")
print(f"Image label: {label_1}")
print(f"Label datatype: {type(label_1)}")

# Checks for class names
class_names = train_data.classes
print(f"Class Names = {class_names}")

# Turn train and test Datasets into DataLoaders
train_dataloader = DataLoader(dataset=train_data, batch_size=16, num_workers=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=16, num_workers=1, shuffle=False)

# Check if the Dataloaders look fine
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of 16")
print(f"Length of test dataloader: {len(test_dataloader)} batches of 16")

# Print to check shape is fine
img_2, label_2 = next(iter(train_dataloader))
print(f"Image shape: {img_2.shape} -> [batch_size, color_channels, height, width]")

# Create a convolutional neural network 
class FaceMask(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(in_channels=3, 
                out_channels=6, 
                kernel_size=3, 
                stride=1, 
                padding=1),

      nn.ReLU(),
      nn.Conv2d(in_channels=6, 
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,
      stride=2) 
)

    self.block_2 = nn.Sequential(
      nn.Conv2d(in_channels=16, out_channels=20, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=20, out_channels=24, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2)
)
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(6*40*40,output_shape)
)
  def forward(self, x: torch.Tensor):
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.classifier(x)
    return x

# Create first model
model_0 = FaceMask(input_shape=3, hidden_units=10, output_shape=len(train_data.classes)).to(device)

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, 
optimizer: torch.optim.Optimizer):
# Put model in train mode
  model.train()
# Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0
# Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
# Send data to target device
    X, y = X.to(device), y.to(device)
# 1. Forward pass
    y_pred = model(X)
# 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item() 
# 3. Optimizer zero grad
    optimizer.zero_grad()
# 4. Loss backward
    loss.backward()
# 5. Optimizer step
    optimizer.step()
# Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
# Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
    dataloader: torch.utils.data.DataLoader, 
    loss_fn: torch.nn.Module):
# Put model in eval mode
  model.eval() 
# Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0
# Turn on no_grad context manager
  with torch.no_grad():
# Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
# Send data to target device
      X, y = X.to(device), y.to(device)
# 1. Forward pass
      test_pred_logits = model(X)
# 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
# Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
# Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Combine train_step & test_step functions
# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer, loss_fn: torch.nn.Module = nn.CrossEntropyLoss(), epochs: int = 5):
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn)        
        # 4. Print loss & accuracies
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )
        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    # 6. Return the filled results at the end of the epochs
    return results

# Set number of epochs
NUM_EPOCHS = 5

# Start the timer
start_time = timer()

model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

def plot_loss_curves(results: Dict[str, List[float]]):
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

plot_loss_curves(model_0_results)

# Save model_0
# Create models directory (if it doesn't already exist)

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # Create parent directories if needed
exist_ok=True # Allows exisiting directory
)
# Create model save path
MODEL_NAME = "model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(), 
f=MODEL_SAVE_PATH)


In [ ]:
# This sets up, creates and trains model 2
import os
import torch, torchvision
from torch import nn
from pathlib import Path
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm
from typing import List, Tuple, Dict

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup path to data folder
data_path = Path("./data/")
image_path = data_path / "Face_Mask_Dataset"
# Setup train, testing and validation paths
train_dir = image_path / "Train"
test_dir = image_path / "Test"
val_dir = image_path / "Validation"

# Traverse through the data directory and list contents
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Traverses and prints directory details
walk_through_dir(image_path)

# Transform (with 1 data aug to improve performance) to tensor
data_transform = transforms.Compose([transforms.Resize(64), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])

# Create pytorch datasets for train, test and validation
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

# Checking the data and a sample
print(f"Train data:\n{train_data}\nTest data:\n{test_data}")
img_1, label_1 = train_data[0][0], train_data[0][1]
print(f"Image tensor:\n{img_1}")
print(f"Image shape: {img_1.shape}")
print(f"Image datatype: {img_1.dtype}")
print(f"Image label: {label_1}")
print(f"Label datatype: {type(label_1)}")

# Checks for class names
class_names = train_data.classes
print(f"Class Names = {class_names}")

# Turn train and test Datasets into DataLoaders
train_dataloader = DataLoader(dataset=train_data, batch_size=16, num_workers=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=16, num_workers=1, shuffle=False)

# Check if the Dataloaders look fine
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of 16")
print(f"Length of test dataloader: {len(test_dataloader)} batches of 16")

# Print to check shape is fine
img_2, label_2 = next(iter(train_dataloader))
print(f"Image shape: {img_2.shape} -> [batch_size, color_channels, height, width]")

# Create a convolutional neural network 
class FaceMask(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(in_channels=3, 
                out_channels=6, 
                kernel_size=3, 
                stride=1, 
                padding=1),

      nn.ReLU(),
      nn.Conv2d(in_channels=6, 
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,
      stride=2) 
)

    self.block_2 = nn.Sequential(
      nn.Conv2d(in_channels=16, out_channels=20, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=20, out_channels=24, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2)
)
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(6*32*32,output_shape)
)
  def forward(self, x: torch.Tensor):
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.classifier(x)
    return x

# Create first model
model_0 = FaceMask(input_shape=3, hidden_units=10, output_shape=len(train_data.classes)).to(device)

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, 
optimizer: torch.optim.Optimizer):
# Put model in train mode
  model.train()
# Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0
# Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
# Send data to target device
    X, y = X.to(device), y.to(device)
# 1. Forward pass
    y_pred = model(X)
# 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item() 
# 3. Optimizer zero grad
    optimizer.zero_grad()
# 4. Loss backward
    loss.backward()
# 5. Optimizer step
    optimizer.step()
# Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
# Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
    dataloader: torch.utils.data.DataLoader, 
    loss_fn: torch.nn.Module):
# Put model in eval mode
  model.eval() 
# Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0
# Turn on no_grad context manager
  with torch.no_grad():
# Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
# Send data to target device
      X, y = X.to(device), y.to(device)
# 1. Forward pass
      test_pred_logits = model(X)
# 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
# Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
# Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Combine train_step & test_step functions
# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer, loss_fn: torch.nn.Module = nn.CrossEntropyLoss(), epochs: int = 5):
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn)        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )
        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    # 6. Return the filled results at the end of the epochs
    return results

# Set number of epochs
NUM_EPOCHS = 5

# Start the timer
start_time = timer()

model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

def plot_loss_curves(results: Dict[str, List[float]]):
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

plot_loss_curves(model_0_results)

# Save model_0
# Create models directory (if it doesn't already exist)

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # Create parent directories if needed
exist_ok=True # Allows exisiting directory
)
# Create model save path
MODEL_NAME = "model_1.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_1.state_dict(), 
f=MODEL_SAVE_PATH)


In [ ]:
# This loads a saved model and performs predictions on the validation set
import os
import torch, torchvision
from torch import nn
from pathlib import Path
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm
from typing import List, Tuple, Dict

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create the same convolutional neural network that trained model_1
class FaceMask(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape, 
                out_channels=hidden_units, 
                kernel_size=3, 
                stride=1, 
                padding=1),

      nn.ReLU(),
      nn.Conv2d(in_channels=6, 
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,
      stride=2) 
)

    self.block_2 = nn.Sequential(
      nn.Conv2d(in_channels=16, out_channels=20, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=20, out_channels=24, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2)
)
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(6*32*32,output_shape)
)
  def forward(self, x: torch.Tensor):
    x = self.block_1(x)
    x = self.block_2(x)    
    x = self.classifier(x)
    return x

loaded_model_1 = FaceMask(input_shape=3, hidden_units=6, output_shape=2)

# Load in the saved state_dict()
MODEL_PATH = Path("models")
MODEL_NAME = "model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
loaded_model_1.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_model_1 = loaded_model_1.to(device)

# Setup path to data folder
data_path = Path("./data/")
image_path = data_path / "Face_Mask_Dataset"
# Setup validation path
val_dir = image_path / "Validation"

# Transform (w/o data aug since validation) to tensor
data_transform = transforms.Compose([transforms.Resize(64), 
transforms.ToTensor()])

# Create pytorch datasets for validation
val_data = datasets.ImageFolder(root=val_dir, transform=data_transform)

# Turn Validation Datasets into DataLoader
val_dataloader = DataLoader(dataset=val_data, batch_size=16, num_workers=1, shuffle=False)

# Define function for predictions
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
    pred_probs = [] #Create an empty list for prediction probabilities 
    model.to(device)
    model.eval()
    with torch.no_grad():
        for sample in data:
            # unsqueeze & add a batch size dimension on the sample
            sample = torch.unsqueeze(sample, dim=0).to(device)
            # Forward pass
            pred_logit = model(sample) #pass the sample to our target model
            # Get prediction probability from a given sample
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0) # since multi-class, use the softmax activation function on our pred logit
            # MOve to cpu for matplotlib
            pred_probs.append(pred_prob.cpu())
    return torch.stack(pred_probs)

# Validation Test - We validate our model against the validation set

test_samples = [] #create empty list for validation data
test_labels = []  # create empty list for corresponding labels
for sample, label in list(val_data): # put correspinding data & label from validation set into above lists
    test_samples.append(sample)
    test_labels.append(label)

# Make predictions on test samples with model 
pred_probs= make_predictions(model=loaded_model_1, data=test_samples)

# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)


count_correct = 0
count_incorrect = 0
for i, sample in enumerate(test_samples):
  # Find the prediction label
  pred_label = val_data.classes[pred_classes[i]]
  # Get the truth label
  truth_label = val_data.classes[test_labels[i]] 
  if pred_label == truth_label:
      count_correct +=1
  else:
      count_incorrect +=1

print('Number of correctly predicted images was:', count_correct,'\nNumber of incorrectly predicted images was:', count_incorrect)
print(f"\nAccuracy = {(count_correct)/8} %") #There are a total of 800 images in validation set

In [ ]:
# This loads a saved model and predicts on a random sample of images from validation set
# This loads the saved model and predicts on validation set
import os
import torch, torchvision
from torch import nn
from pathlib import Path
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm
from typing import List, Tuple, Dict
import random

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create the same convolutional neural network that trained model_0
class FaceMask(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape, 
                out_channels=hidden_units, 
                kernel_size=3, # how big is the square that's going over the image?
                stride=1, # default
                padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int 

      nn.ReLU(),
      nn.Conv2d(in_channels=6, 
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,
      stride=2) # default stride value is same as kernel_size
)

    self.block_2 = nn.Sequential(
      nn.Conv2d(in_channels=16, out_channels=20, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=20, out_channels=24, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2)
)
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(6*32*32,output_shape)
)
  def forward(self, x: torch.Tensor):
    x = self.block_1(x)
    x = self.block_2(x)    
    x = self.classifier(x)
    return x

loaded_model_1 = FaceMask(input_shape=3, hidden_units=6, output_shape=2)

# Load in the saved state_dict()
MODEL_PATH = Path("models")
MODEL_NAME = "model_2.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
loaded_model_1.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_model_1 = loaded_model_1.to(device)

# Setup path to data folder
data_path = Path("./data/")
image_path = data_path / "Face_Mask_Dataset"
# Setup validation path
val_dir = image_path / "Validation"

# Transform (w/o data aug since validation) to tensor
data_transform = transforms.Compose([transforms.Resize(64), 
transforms.ToTensor()])

# Create pytorch datasets for validation
val_data = datasets.ImageFolder(root=val_dir, transform=data_transform)

# Turn Validation Datasets into DataLoader
val_dataloader = DataLoader(dataset=val_data, batch_size=16, num_workers=1, shuffle=False)

# Define function for predictions
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
    pred_probs = [] #Create an empty list for prediction probabilities 
    model.to(device)
    model.eval()
    with torch.no_grad():
        for sample in data:
            # unsqueeze & add a batch size dimension on the sample
            sample = torch.unsqueeze(sample, dim=0).to(device)
            # Forward pass
            pred_logit = model(sample) #pass the sample to our target model
            # Get prediction probability from a given sample
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0) # since multi-class, use the softmax activation function on our pred logit
            # MOve to cpu for matplotlib
            pred_probs.append(pred_prob.cpu())
    return torch.stack(pred_probs)

test_samples = []
test_labels = []
for sample, label in random.sample(list(val_data), k=25): # to randomly sample 25 images from validation set
    test_samples.append(sample)
    test_labels.append(label)

# Make predictions on test samples with model 1
pred_probs= make_predictions(model=loaded_model_1, data=test_samples)

# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)

# Plot predictions
plt.figure(figsize=(25, 25))
nrows = 9
ncols = 3
for i, sample in enumerate(test_samples):
  # Create a subplot
  plt.subplot(nrows, ncols, i+1)

  # Plot the target image
  plt.imshow(sample.squeeze().permute(1, 2, 0), cmap="gray")

  # Find the prediction label
  pred_label = val_data.classes[pred_classes[i]]

  # Get the truth label
  truth_label = val_data.classes[test_labels[i]] 

  # Create the title text of the plot
  title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
  if pred_label == truth_label:
      plt.title(title_text, fontsize=10, c="g", pad=3) # green text if correct
  else:
      plt.title(title_text, fontsize=10, c="r", pad=3) # red text if wrong
  plt.axis(False)
plt.show()